## CSCI 183 Data Science
### Spam Filtering for Short Messages: Features Matrix
#### Ryan Johnson, Grace Nguyen, and Raya Young




In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import math
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

#### Import the test data 
Separate into two arrays: spam and ham. These arrays will be processed individually in order to generate word clouds.

In [2]:
data = pd.read_csv("training-data/spamcollectiondata.tsv", sep='\t', names = ["Category", "Message"])
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


#### Converting to lowercase

In [3]:
message_data = [word.lower() for word in data['Message']]
category = data['Category'].tolist()

#### Stopword removal and Stemming
Clean both sets of data by removing stopwords. This way, the word cloud will not be completely populated by common stop words. Stemming is also important to ensure eliminate the possibility of having multiple different forms of words.


In [4]:
stop = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')
training_set = []
i = 0
for message in message_data:
    sentence = message.split(" ")
    filtered = []
    pr = []
    for word in sentence:
        if word.lower() not in stop:
            stemmed = stemmer.stem(word)
            filtered.append(stemmed)
    pr.append(filtered)
    pr.append(category[i])
    training_set.append(pr)
    i = i+1
    
#print first 10 elements in training_set to see format
print(training_set[:10])

[[['go', 'jurong', 'point,', 'crazy..', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'cine', 'got', 'amor', 'wat...'], 'ham'], [['ok', 'lar...', 'joke', 'wif', 'u', 'oni...'], 'ham'], [['free', 'entri', '2', 'wkli', 'comp', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005.', 'text', 'fa', '87121', 'receiv', 'entri', 'question(std', 'txt', 'rate)t&c', 'appli', '08452810075over18'], 'spam'], [['u', 'dun', 'say', 'earli', 'hor...', 'u', 'c', 'alreadi', 'say...'], 'ham'], [['nah', "don't", 'think', 'goe', 'usf,', 'live', 'around', 'though'], 'ham'], [['freemsg', 'hey', 'darl', 'it', '3', 'week', 'word', 'back!', "i'd", 'like', 'fun', 'still?', 'tb', 'ok!', 'xxx', 'std', 'chgs', 'send,', '£1.50', 'rcv'], 'spam'], [['even', 'brother', 'like', 'speak', 'me.', 'treat', 'like', 'aid', 'patent.'], 'ham'], [['per', 'request', 'mell', 'mell', '(oru', 'minnaminungint', 'nurungu', 'vettam)', 'set', 'callertun', 'callers.', 'press', '*9', 'copi', 'friend', 'callertun'], 'h

#### Convert training_set to Dictionary

NLTK's classifiers only accept texts in hashable formats, so we need to convert our list into a dictionary

In [5]:
def list_to_dict(words_list):
  return dict([(word, True) for word in words_list])
 
training_set_formatted = [(list_to_dict(element[0]), element[1]) for element in training_set]

Convert dictionary to a X matrix and Y vector

In [6]:
def generate_words_vector(training_set):
    words_vector = [] 
    for review in training_set:
        for word in review[0]:
            if word not in words_vector: words_vector.append(word) 
    return words_vector
 
def generate_Y_vector(training_set, training_class):
    no_reviews = len(training_set)
    Y = np.zeros(shape=no_reviews)
    for ii in range(0,no_reviews):
        review_class = training_set[ii][1]
        Y[ii] = 1 if review_class == training_class else 0
    return Y
        
def generate_X_matrix(training_set, words_vector):
    no_reviews = len(training_set)
    no_words = len(words_vector)
    X = np.zeros(shape=(no_reviews, no_words+1))
    for ii in range(0,no_reviews):
        X[ii][0] = 1
        review_text = training_set[ii][0]
        total_words_in_review = len(review_text)
        rt = list(review_text)
        for word in rt:
            word_occurences = rt.count(word)
            word_index = words_vector.index(word)+1
            X[ii][word_index] = word_occurences / float(total_words_in_review)
    return X

words_vector = generate_words_vector(training_set_formatted)
X = generate_X_matrix(training_set_formatted, words_vector)
Y_neg = generate_Y_vector(training_set_formatted, 'neg')

Perform Logistic Regression on the features

In [9]:
# Sigmoid Function
# @param x                 double

def sigmoidFunction(a):
    
    return 1 / (1 + np.exp(-a))

#################################################################################

# Cost Function
# @param X                 nxm Matrix
# @param y                 1xm Vector
# @param theta             1xn Vector
# @return

def computeCost(X, y, theta):
    
    m = len(y)
    
    # theta * X    (1xn * nxm = 1xm)
    hypothesis = (np.array(theta) * np.array(X)).tolist()

    # h = sigmoid(theta * X)
    for idx in range(m):
        hypothesis[idx] = sigmoidFunction(hypothesis[idx])

    # log(h) * y' + log(1 - h) * (1 - y)'
    sum = 0
    for idx in range(m):
        sum += y[idx] * math.log(hypothesis[idx]) + (1 - y[idx]) * math.log(1 - hypothesis[idx])

    # -(log(h) * y' + log(1 - h) * (1 - y)') / m
    return -sum / m

#################################################################################

# Gradient Descent
# @param X                 nxm Matrix
# @param y                 1xm Vector
# @param theta             1xn Vector
# @param alpha             step factor
# @param maxInteration     Maximum iteration for convergence
# @return

def gradientDescent(X, y, theta, alpha, maxIteration):
    
    n = len(X)
    m = len(y)
    
    # size: maxIteration
    costHistory = []

    for it in range(maxIteration):

        # size: n
        temp_theta = np.zeros(n)

        # size: m
        hypothesis = np.zeros(m)

        # get values for hypothesis vector
        for i in range(m-1):
            
            # g_value = theta^T * X[i] <-- dot product
            g_value = np.dot(np.array(theta), np.array(X[i]))

            #g_value = 0
            #for j in range(n-1):
            #    g_value += theta[j]*X[j][i]
            
            hypothesis[i] = sigmoidFunction(g_value)

        # for each feature, do gradient descent
        for i in range(n):

            #sum = 0
            tempsum = np.dot(np.array(hypothesis)-np.array(y),np.array(X[i]))
            #for j in range(m-1):
             #   sum += (hypothesis[j] - y[j])*X[i][j]

            temp_theta[i] = theta[i] - (alpha/m)*tempsum
            print(tempsum)

        # update theta
        for i in range(n-1):
            theta[i] = temp_theta[i]

        # compute and record cost
        costHistory[it] = computeCost(X, y, theta)
        print("Done iteration: {}. Current Cost: {}.".format(it,costHistory[it]))

    return costHistory

Put our new X matrix and Y vector into the gradient descent function

In [10]:
theta = np.zeros(len(X))
X = np.matrix(X)
X = np.transpose(X)
X = X.tolist()
gradientDescent(X,Y_neg,theta,.01,100)

ValueError: shapes (5572,) and (12236,) not aligned: 5572 (dim 0) != 12236 (dim 0)

In [41]:
print(len(theta))

5572


In [3]:
import sys
import pylab as pl
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib as matplot
import numpy as np
from scipy.optimize import fmin_bfgs
from sklearn import linear_model

class IrisDatasetLogisticRegression:
        """
         Implementing logistic regression using Iris dataset
        """     
        
        """Global class variables"""
        "Matrix containing set of features"
        X = None
        
        "Matrix containing set of outputs"
        y = None
        
        def __init__(self, X, y):
            """ USAGE:
            Default constructor            
           
            PARAMETERS:
            X - feature matrix
            y - output matrix     
            
            RETURN:            
            """          
            self.X = X
            self.y = y            
            
            """Convert y into a proper 2 dimensional array/matrix. This is to facilitate proper matrix arithmetics."""
            if len(y.shape) == 1:
                y.shape = (y.shape[0],1)
            
                    
        def sigmoid(self,z):
            """ USAGE:
            Compute the sigmoid of each value of z (z can be a matrix, vector or scalar).            
           
            PARAMETERS:
            z - Matrix, vector or scalar       
            
            RETURN:
            The sigmoid value
            """    
            return 1.0 / (1.0 + np.exp(-z))
        
        
        def compute_cost(self,X, y, theta):
            """ USAGE:
            Define the cost function           
          
            PARAMETERS:
            X - Features
            y - Output
            theta        
           
            RETURN:
            return the cost function value
            """    
            m = X.shape[0]
            z = np.dot(X,theta)            
            h = self.sigmoid(z);    
           
            J=(float(-1)/m)*((y.T.dot(np.log(h))) + ((1 - y.T).dot(np.log(1 - h))))           
            return J
        
        
        def compute_gradient(self,X, y, theta):
            """ USAGE:                  
            Compute the gradient using vectorization.
            
            PARAMETERS:           
            X - Features
            y - Output
            theta 
            
            RETURN:           
            """    
            m = X.shape[0]
            z = np.dot(X,theta)            
            h = self.sigmoid(z);
    
            grad = (float(1)/m)*((h-y).T.dot(X))          
            return grad
        

        def plot_two_features(self):
            """ USAGE:
            Plot first two features from the Iris dataset  
          
            PARAMETERS:           
            
            RETURN:    
            """      
            fig = plt.figure()
            ax = fig.add_subplot(111, title ="Iris Dataset - Plotting two features", xlabel = 'Sepal Length', ylabel='Sepal Width')
            plt.setp(ax.get_xticklabels(), visible=False)
            plt.setp(ax.get_yticklabels(), visible=False)
                     
                        
            setosa = np.where(self.y == 0)
            versicolour = np.where(self.y ==1)
            
            ax.scatter(X[setosa, 0], X[setosa, 1], s=20, c='r', marker ='o')  
            ax.scatter(X[versicolour, 0], X[versicolour, 1], s=20, c='r', marker ='x') 
            plt.legend( ('Iris Type - Setosa', 'Iris Type - Versicolour') )
            plt.show()
            

        def plot_three_features(self):
            """ USAGE:
            Plot first two features from the Iris dataset  
          
            PARAMETERS:           
            
            RETURN:    
            """      
            fig = plt.figure()
            ax = fig.add_subplot(111, title ="Iris Dataset - Plotting three features", xlabel = 'Sepal Length', ylabel='Sepal Width', zlabel='Petal Length', projection='3d')
            plt.setp(ax.get_xticklabels(), visible=False)
            plt.setp(ax.get_yticklabels(), visible=False)
            plt.setp(ax.get_zticklabels(), visible=False)
           
            ax.scatter(X[:, 0], X[:, 1],X[:, 2], s=20, c='r', marker ='o')                   
            plt.show()
            
            
        def run_logistic_regression(self):
            """ USAGE:
            Apply principles of logistic regression
          
            PARAMETERS:           
            
            RETURN:    
            """  
            
            """m= number of training data, n= number of features"""
            m = X.shape[0]
            n = X.shape[1]
            
            """Add intercept term to X"""
            self.X = np.hstack((np.ones((m, 1)), self.X))
            
            
            """Initialize fitting parameters. Take into account the intercept term."""
            initial_theta = np.zeros((n + 1, 1))
            
            """"Compute initial cost and gradient"""
            cost = self.compute_cost(self.X, self.y, initial_theta)            
            gradient = self.compute_gradient(self.X, self.y, initial_theta)
            
            print ('Cost at initial theta (zeros): {0} \nGradient at initial theta (zeros): {1}'.format(cost, gradient))
        
            def f(theta):
                return np.ndarray.flatten(self.compute_cost(self.X, self.y, initial_theta))
            
            def fprime(theta):
                return np.ndarray.flatten(self.compute_gradient(self.X, self.y, initial_theta))
            
            print(fmin_bfgs(f, initial_theta, fprime, disp=True, maxiter=400, full_output = True, retall=True))        
          
           
try:
#    iris = datasets.load_iris()
#    X = iris.data
#    Y = iris.target
    
    data = np.loadtxt('data/ex2data1.txt', delimiter=',')
    X = data[:, 0:2]   
    Y = data[:, 2]
   
    
    logistic_regression  = IrisDatasetLogisticRegression(X, Y)
    #logistic_regression.plot_two_features()
    #logistic_regression.plot_three_features()
    
    logistic_regression.run_logistic_regression()
    
except:
    print ("unexpected error:", sys.exc_info())

unexpected error: (<class 'FileNotFoundError'>, FileNotFoundError(2, 'No such file or directory'), <traceback object at 0x108f9dd88>)
